# Text preprocessing techniques for sequence models

To feed a text dataset to an LSTM in TensorFlow, you will typically need to follow these steps:

1. **Tokenization:** Convert the raw text into a sequence of tokens, which are usually words or subwords. Tokenization can be performed using various libraries such as the NLTK, Spacy, or TensorFlow Tokenizer.

2. **Vocabulary creation:** Create a vocabulary by mapping each unique token to a unique integer index. The size of the vocabulary will depend on the number of unique tokens in your dataset.

3. **Text encoding:** Encode each token sequence as a sequence of integer indices using the vocabulary mapping. This step converts the text data into a numerical format that can be fed into the LSTM. 

4. **Padding:** Since the input sequences may have different lengths, it is common to pad the sequences with zeros to make them all the same length. 

5. **Creating input and target sequences:** The LSTM requires input and target sequences to be specified. The input sequences are typically a subset of the encoded token sequences, shifted by one time step. The target sequences are the same as the encoded token sequences, shifted by one time step. 

6. **Batching:** Group the input and target sequences into batches for efficient processing by the LSTM. 

7. **Model training:** Train the LSTM model using the input and target sequences. 

When feeding text data into an LSTM, the input data is typically in the form of a sequence of integers, where each integer correspondsto a specific token in the vocabulary. The LSTM expects the input data to be in the form of a 3D tensor with shape `(batch_size, sequence_length, input_dim)`, where `batch_size` is the number of sequences in each batch, `sequence_length` is the length of each sequence (after padding), and `input_dim` is the dimensionality of the input at each time step. 

For text data, `input_dim` is usually set to the size of the vocabulary (i.e., the number of unique tokens), since each token is represented by a one-hot encoding or an embedding vector. Alternatively, you can use pre-trained word embeddings such as GloVe or Word2Vec, which map each token to a high-dimensional vector that captures its semantic meaning.

In TensorFlow, you can use the `tf.keras.preprocessing.text.Tokenizer` class to perform tokenization, vocabulary creation, and text encoding. In the following, you can see an example:

Once the input data has been prepared, you can create an LSTM model using `tf.keras.layers.LSTM` and train it using the input and target sequences.

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define the text data
texts = ['This is the first sentence', 'This is the second sentence', 'This is the third sentence which has more words']

# Create a tokenizer and fit it on the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)


In [5]:

# Encode the text data as a sequence of integer indices
sequences = tokenizer.texts_to_sequences(texts)


In [3]:
sequences

[[1, 2, 3, 5, 4], [1, 2, 3, 6, 4], [1, 2, 3, 7, 4, 8, 9, 10, 11]]

In [6]:

# Pad the sequences to make them all the same length
padded_sequences = pad_sequences(sequences, maxlen=7, padding='post')


In [7]:
padded_sequences

array([[ 1,  2,  3,  5,  4,  0,  0],
       [ 1,  2,  3,  6,  4,  0,  0],
       [ 3,  7,  4,  8,  9, 10, 11]], dtype=int32)

In [ ]:
padded_sequences.shape

(3, 7)

Note that the `maxlen` parameter in `pad_sequences` specifies the maximum length of the padded sequences. Any sequences longer than this will be truncated, while shorter sequences will be padded with zeros. 

The `padding` parameter specifies whether to pad the sequences before or after the actual sequence data. `padding='post'` is an argument in the `pad_sequences` function that specifies where to pad the sequences. When `padding='post'`, the function adds zeros to the end of each sequence, after the actual sequence data. 

For example, suppose we have two sequences: `[1, 2, 3]` and `[1, 2, 3, 4, 5]`, and we want to pad them to a length of 7. If we use `padding='post'`, the padded sequences will be:

```
[1, 2, 3, 0, 0, 0, 0]
[1, 2, 3, 4, 5, 0, 0]
```

Note that the zeros are added to the end of each sequence, after the actual sequence data. 

If we use `padding='pre'` instead, the padded sequences will be:

```
[0, 0, 0, 1, 2, 3, 0]
[0, 0, 1, 2, 3, 4, 5]
```

Note that the zeros are added to the beginning of the first sequence, and to the end of the second sequence, to make them both the same length. 

The choice of `padding` depends on the specific requirements of your model. In some cases, padding at the beginning of the sequencesmay be more appropriate, while in other cases padding at the end may be better. For example, when processing text data, it is common to use `padding='post'` because it preserves the order of the words in the sentence. On the other hand, when processing time series data, it may be more appropriate to use `padding='pre'` because it aligns the sequences with the time axis. In general, you should choose the padding strategy that makes the most sense for your specific use case.

# Embedding Layer

The main functionality of the Keras Embedding layer is to create a dense vector representation of each word in a given text corpus. It takes as input a sequence of integer-encoded words, where each integer represents a specific word in the vocabulary of the corpus. It then maps each integer to a corresponding dense vector representation of fixed size. This is achieved through a process called embedding, where each word is represented by a dense vector of real numbers, typically of a lower dimension than the size of the vocabulary.

The embedding layer assigns a unique vector to each word, where the vectors are learned during the training process. The embedding layer can be considered as a lookup table that maps each word in the input sequence to its corresponding dense vector representation. These dense vectors capture semantic similarity between words, such that similar words have vectors that are closer together in the vector space.

The output of the embedding layer is a 3D tensor of shape (batch_size, sequence_length, output_dimension), where `batch_size` is the number of samples in each batch, `sequence_length` is the length of the input sequence, and `output_dimension` is the dimension of the dense vector representation of each word. This output can then be fed into other layers of the neural network for further processing, such as a recurrent or convolutional layer.
I think this online reference is informative: https://medium.com/analytics-vidhya/understanding-embedding-layer-in-keras-bbe3ff1327ce

In [14]:
import tensorflow as tf

# Define the embedding layer
vocab_size = 12 #since we want to embed integers between 0 and 11 
embedding_dim = 3 #the size of output vector for embedding each integer
embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=10)

# Compute the output of the embedding layer
embedding_output = embedding_layer(padded_sequences)

# Print the output tensor
print(embedding_output)

tf.Tensor(
[[[-0.01084356  0.03168121 -0.00521006]
  [ 0.049929    0.02468925  0.04158319]
  [ 0.02804733 -0.02485967 -0.0365837 ]
  [-0.01609825 -0.0162029   0.00502186]
  [-0.02367218 -0.02722814  0.01044212]
  [-0.00195447  0.02289129  0.01561144]
  [-0.00195447  0.02289129  0.01561144]]

 [[-0.01084356  0.03168121 -0.00521006]
  [ 0.049929    0.02468925  0.04158319]
  [ 0.02804733 -0.02485967 -0.0365837 ]
  [-0.02004063  0.00764884 -0.04887904]
  [-0.02367218 -0.02722814  0.01044212]
  [-0.00195447  0.02289129  0.01561144]
  [-0.00195447  0.02289129  0.01561144]]

 [[ 0.02804733 -0.02485967 -0.0365837 ]
  [ 0.02378476 -0.04357032  0.01154632]
  [-0.02367218 -0.02722814  0.01044212]
  [ 0.04723329 -0.00957887 -0.03587203]
  [-0.0077972  -0.04284241 -0.01361668]
  [-0.03137993  0.03929946 -0.01355876]
  [-0.04841976 -0.00267458 -0.03714458]]], shape=(3, 7, 3), dtype=float32)


## Parameters of Tokenizer method

The Tokenizer class has various parameters that can be set to control its behavior. Here's what the num_words and oov_token parameters mean:

num_words: This parameter specifies the maximum number of words to keep in the vocabulary. Only the most frequent num_words words will be kept, based on their frequency in the training data. Any less frequent words will be discarded. This helps to limit the vocabulary size and reduce the dimensionality of the input data. In the example you provided, num_words=5 means that only the top 5 most frequent words will be kept in the vocabulary.

oov_token: This parameter is used to specify a token that will be used to represent any out-of-vocabulary (OOV) words in the input data. OOV words are words that are not present in the vocabulary learned during training. By default, OOV words are represented as 1, but you can specify a custom token using the oov_token parameter. In the example you provided, oov_token='<OOV>' means that the string <OOV> will be used to represent any OOV words in the input data.

In [ ]:
# Define the text data
texts = ['This is the first sentence', 'This is the second sentence', 'This is the third sentence which has more words']

tokenizer = Tokenizer(num_words=5, oov_token='<OOV>')
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)

sequences

[[2, 3, 4, 1, 1], [2, 3, 4, 1, 1], [2, 3, 4, 1, 1, 1, 1, 1, 1]]